In [ ]:
! pip install pmdarima

In [ ]:
from pmdarima import auto_arima

In [ ]:
# ! pip install yfinance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import pandas as pd 

In [ ]:
# df = yf.download('tcs',start='2022-03-30',end='2023-03-30')
# f = open('stocksdata_acc.csv','w')
# df.to_csv('stocksdata_acc.csv')
# f.close()
df = pd.read_csv('stocksdata_acc.csv')

In [ ]:
# df = pd.read_excel('MCAP31032023_0.xlsx')
# sym_comp = df['Symbol']
# sym_comp.dropna(inplace=True)
# sym_comp.isna().sum()
# tickers = []
# tickers.extend(sym_comp.values)

In [ ]:
# df = yf.download('tcs',start='2023-03-31',end = '2023-06-01')
# f = open('stocksdata_test.csv','w')
# df.to_csv('stocksdata_test.csv')
# f.close()

In [ ]:
tcs_test = pd.read_csv('stocksdata_test.csv')
dic = {'year':[],'month':[],'date':[]}
lth = len(tcs_test.Date)
for i in range(lth):
    lst = tcs_test.Date[i].split(sep='-')
    dic['date'].append(lst[2])
    dic['month'].append(lst[1])
    dic['year'].append(lst[0])

dft = pd.DataFrame(dic)
dft.index
tcs_test['Date'] = pd.to_datetime(dft['date'].astype(str)+'-'+dft['month'].astype(str)+'-'+dft['year'].astype(str),dayfirst=True,infer_datetime_format=True) # feature type is not datetime 
print(tcs_test.info())
tcs_test.set_index('Date',inplace=True)
print(tcs_test.head())
tcs_test.tail()

In [ ]:
# df = yf.download(tickers[1],period='6mo')
# f = open('stocksdata.csv','w')
# df.to_csv('stocksdata.csv')
# f.close()
df = pd.read_csv('stocksdata_acc.csv')
df

In [ ]:
dic = {'year':[],'month':[],'date':[]}
lth = len(df.Date)
for i in range(lth):
    lst = df.Date[i].split(sep='-')
    dic['date'].append(lst[2])
    dic['month'].append(lst[1])
    dic['year'].append(lst[0])


In [ ]:
dft = pd.DataFrame(dic)
dft.index

In [ ]:
df['Date'] = pd.to_datetime(dft['date'].astype(str)+'-'+dft['month'].astype(str)+'-'+dft['year'].astype(str),dayfirst=True,infer_datetime_format=True) # feature type is not datetime 
print(df.info())

## __Time_Series_Analysis__

In [ ]:
df.set_index('Date',inplace=True)
# print(df.head())
df.tail()

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(data=df,x=df.index,y=df['Close'])
plt.show()

In [ ]:
# calcaulate the plot of rolling mean and standard deviation
df['rollMean'] = df['Close'].rolling(window=5).mean() ## window = 12 parameter shiftes the time by 12 column
df['rollStd'] = df['Close'].rolling(window=5).std()

In [ ]:
print(df['rollMean'])
print(df['rollStd']) # it will take rolling mean after first 12 values

In [ ]:
df.head(5)

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=df,x=df.index,y=df['Close'])
sns.lineplot(data=df,x=df.index,y=df['rollMean'])
sns.lineplot(data=df,x=df.index,y=df['rollStd'])
plt.show()

In [ ]:
# conclusion : Mean is not stationary , SD is not stationary; so our data is not stationary
# Now let us perform the ADF test 

In [ ]:
from statsmodels.tsa.stattools import adfuller
adfTest = adfuller(df['Close'])
adfTest
# Interpret these value below by converting into series

In [ ]:
stats = pd.Series(adfTest[0:4],index=['Test Statistics','p-value','#lags used','number of observation'])
stats

In [ ]:
for key, values in adfTest[4].items():
    print('critically',key,':',values)

In [ ]:
def test_stationary(df,var):
    df['rollMean'] = df[var].rolling(window=12).mean() ## window = 12 parameter shiftes the time by 12 column
    df['rollStd'] = df[var].rolling(window=12).std()
    from statsmodels.tsa.stattools import adfuller
    adfTest = adfuller(df[var])
    stats = pd.Series(adfTest[0:4],index=['Test Statistics','p-value','#lags used','number of observation'])
    print(stats)
    for key, values in adfTest[4].items():
        print('critically',key,':',values)
    plt.figure(figsize=(10,5))
    sns.lineplot(data=df,x=df.index,y=var)
    sns.lineplot(data=df,x=df.index,y='rollMean')
    sns.lineplot(data=df,x=df.index,y='rollStd')
    plt.show()

In [ ]:
tcs_df = df[['Close']].copy() # double bracket cuz its list within a list
tcs_df.head()

In [ ]:
def shift(x):
    tcs_df = df[['Close']].copy()
    tcs_df['shift'] = tcs_df.Close.shift(x) # create a new column which will contain shifted value from Close 
    tcs_df['shiftDiff'] = tcs_df['Close'] - tcs_df['shift']
    # print(tcs_df.sample(20))
    shifted_Data = tcs_df.dropna()
    return shifted_Data

In [ ]:
plt.figure(figsize=(10,5))
sns.lineplot(data=df,x=df.index,y=df['Close'])
sns.lineplot(data=df,x=df.index,y=df['rollMean'])
sns.lineplot(data=df,x=df.index,y=df['rollStd'])
plt.show()

In [ ]:
shifted_Data = shift(2)
test_stationary(shifted_Data,'shiftDiff')

# Interpret these value below by converting into series

In [ ]:
model = auto_arima(tcs_df['Close'],start_p=1,start_q=1,max_p=5,max_q=5,m=12,start_P=0,
                   seasonal=True,d = None,D = 0,trace=True,error_action='ignore',
                   suppress_warnings = True,stepwise = True)

model.summary()

### __ARIMA__

In [ ]:
tcs = df[['Close']].copy(deep=True)
tcs['FirstDiff'] = tcs[['Close']].diff(1) # for ARIMA
#tcs['FirstDiff'] = tcs[['Close']].diff(12) # for

In [ ]:
print(tcs.head())

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
plot_pacf(tcs['FirstDiff'].dropna(),lags=40)
plt.show()

* the shaded area is insignificant area
* PACF gives us the auto regressive values
* frist 'p' is 1 (the axis co-ordinate), whoes value is ~0.31 (the y-axis co-ordinate)
* so, significant P values are 1 ,2 ,3, 4, etc  

In [ ]:
plot_acf(tcs['FirstDiff'].dropna(),lags=40)

In [ ]:
tcs.sort_index()

In [ ]:
import itertools
import statsmodels.api as sm
p=d=q=range(5)

pdq = list(itertools.product(p,d,q))
lst = {}
lst1 = {}
for param in pdq:
    try:
        model_arima = ARIMA(tcs['Close'],order=param)
        model_arima_fit = model_arima.fit()
        lst[param] = model_arima_fit.aic
        lst1[param] = sm.stats.durbin_watson(model_arima_fit.resid.values)
    except:
        continue
x = 0
pdq = 0
for k in lst.keys():
    if x > lst[k]:
        x = lst[k]
        pdq = k


        

In [ ]:
pdq

In [ ]:
plt.figure(figsize=(20,7))
model = ARIMA(tcs['Close'],order=(2,1,3)) # Parameters: p, d, q
model_fit = model.fit()
# prediction = model_fit.predict(start=test.index[0],end=test.index[-1], freq = None)
prediction = model_fit.get_forecast(steps = 15 , exog = df.iloc[-30:]['Close']);
tcs['arimaPred'] = prediction
# print(tcs.tail())

# Plot

sns.lineplot(data=tcs,x=tcs.index,y='Close')
# sns.lineplot(data=tcs,x=tcs.index,y='arimaPred')
plt.plot(pd.date_range(start = df.index[-1],periods = 15 , freq = 'd'),prediction.predicted_mean , label = "test",color = 'Green')
sns.lineplot(data=tcs_test,x=tcs_test.index,y='Close')
plt.show()

# Conclusion: The ARIMA prediction is not good


tcs.dropna()
print('')



In [ ]:
import statsmodels.api as sm

sm.stats.durbin_watson(model_fit.resid.values)

In [45]:
import itertools
p=d=q=range(5)

pdq = list(itertools.product(p,d,q))
lst = {}
for i in pdq:
    try:
        model_arima = ARIMA(tcs['Close'],order=param)
        model_arima_fit = model_arima.fit()
        lst[i]=sm.stats.durbin_watson(model_arima_fit.resid.values)
    except:
        continue

lst

/home/dai/anaconda3/lib/python3.11/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/home/dai/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/dai/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/dai/anaconda3/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates

{(0, 0, 0): 2.9407325608111523,
 (0, 0, 1): 2.9407325608111523,
 (0, 0, 2): 2.9407325608111523,
 (0, 0, 3): 2.9407325608111523,
 (0, 0, 4): 2.9407325608111523,
 (0, 1, 0): 2.9407325608111523,
 (0, 1, 1): 2.9407325608111523,
 (0, 1, 2): 2.9407325608111523,
 (0, 1, 3): 2.9407325608111523,
 (0, 1, 4): 2.9407325608111523,
 (0, 2, 0): 2.9407325608111523,
 (0, 2, 1): 2.9407325608111523,
 (0, 2, 2): 2.9407325608111523,
 (0, 2, 3): 2.9407325608111523,
 (0, 2, 4): 2.9407325608111523,
 (0, 3, 0): 2.9407325608111523,
 (0, 3, 1): 2.9407325608111523,
 (0, 3, 2): 2.9407325608111523,
 (0, 3, 3): 2.9407325608111523,
 (0, 3, 4): 2.9407325608111523,
 (0, 4, 0): 2.9407325608111523,
 (0, 4, 1): 2.9407325608111523,
 (0, 4, 2): 2.9407325608111523,
 (0, 4, 3): 2.9407325608111523,
 (0, 4, 4): 2.9407325608111523,
 (1, 0, 0): 2.9407325608111523,
 (1, 0, 1): 2.9407325608111523,
 (1, 0, 2): 2.9407325608111523,
 (1, 0, 3): 2.9407325608111523,
 (1, 0, 4): 2.9407325608111523,
 (1, 1, 0): 2.9407325608111523,
 (1, 1, 

In [ ]:
lst

In [ ]:
tcs = df[['Close']].copy(deep=True)
print(tcs.head())

# Create columns for one month and one year lagged data
tcs['firstDiff'] = tcs['Close'].diff()
tcs['Diff5'] = tcs['Close'].diff(5) # This will show nulls to start with

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_pacf(tcs['Diff5'].dropna(),lags=40)
plt.show() 

plot_acf(tcs['Diff5'].dropna(),lags=40)
plt.show()

In [ ]:
# train = tcs[:round(len(tcs)*70/100)] # Take the first 70% data
# test = tcs[round(len(tcs)*70/100):] # Take the last 30% data, starting from 71%

# First ARIMA prediction
# model = ARIMA(train['Clorediction = model_fit.predict(start=test.index[0],end=test.index[-1])
# tcs['sarimaxPred'] = prediction
# print(tcs.tail()),order=(3,1,3)) # Parameters: p, d, q
# model_fit = model.fit()
# prediction = model_fit.predict(start=test.index[0],end=test.index[-1])
# tcs['arimaPred'] = prediction

# Now SARIMAX prediction
model = SARIMAX(tcs['Close'],order=(0,0,0), seasonal_order=(0,1,0,12))
model_fit = model.fit()
# prediction = model_fit.predict(start=test.index[1],end=test.index[-1])
prediction = model_fit.get_forecast(steps = 30 , exog = df.iloc[-30:]['Close'])
tcs['sarimaxPred'] = prediction
# print(tcs.tail())


In [ ]:
import statsmodels.api as sm

sm.stats.durbin_watson(model_fit.resid.values)

In [ ]:

# Plot
futureDate = pd.DataFrame(pd.date_range(start='2023-01-07', end='2023-10-09',freq='D'),columns=['Dates'])
predict = model_fit.predict(start=futureDate.index[0],end=futureDate.index[-1])
tcs.dropna()
plt.figure(figsize=(15,6))
sns.lineplot(data=tcs_test,x=tcs_test.index,y='Close')
sns.lineplot(data=tcs,x=tcs.index,y='Close')
plt.plot(pd.date_range(start = df.index[-1],periods = 30 , freq = 'd'),prediction.predicted_mean , label = "test",color = 'Black');
# sns.lineplot(data=tcs,x=tcs.index,y='arimaPred')
# model_fit.predict(start=futureDate.index[0],end=futureDate.index[-1],step = 30).plot(color='black')
plt.show()


In [ ]:
# Compared to ARIMA, SARIMAX is much better

# Future prediction

# First check the last date in our dataset
# print(tcs.tail())

# MS: Month Start frequency
# Create a data frame to hold index values from 01.01.61 to 01.12.62
futureDate = pd.DataFrame(pd.date_range(start='2023-01-07', end='2023-10-09',freq='D'),columns=['Dates'])
# futureDate.set_index('Dates',inplace=True)
# print(futureDate.head())

# Predict and print
predict = model_fit.predict(start=futureDate.index[0],end=futureDate.index[-1])
# print(predict)